# Linearization: Design Study C

In [2]:
import sympy
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dotprint(expr):
    display(Math(vlatex(expr)))

In [3]:
t = symbols('t')

# Generalized Coordinates
theta, phi = symbols(r'theta, phi', cls=Function)
theta = theta(t)
phi = phi(t)

theta_dot = theta.diff(t)
phi_dot = phi.diff(t)

theta_ddot = theta.diff(t,2)
phi_ddot = phi.diff(t,2)

Js, Jp, k, tau_theta, b  = symbols(r'J_s, J_p, k, tau_\theta, b', real=True)

In [4]:
LHS = Matrix([
    Js*theta_ddot+b*(theta_dot-phi_dot)+k*(theta-phi),
    Jp*phi_ddot+b*(phi_dot-theta_dot)+k*(phi-theta)
])
RHS = Matrix([
    tau_theta,
    0
])

dynamics = Eq(LHS, RHS)
dotprint(dynamics)

<IPython.core.display.Math object>

## Deriving Nonlinear State Space Equations

Solve for our highest derivatives, $\ddot{\phi}, \ddot{\theta}$:

In [5]:
solve_dict = solve(dynamics, (theta_ddot, phi_ddot), simplify=True, dict=True)[0]
dotprint(solve_dict)

<IPython.core.display.Math object>

Create our state vector $x = [x_1, x_2, x_3, x_4]$:

In [6]:
state = MatrixSymbol('x', 4,1)
dotprint(state)

<IPython.core.display.Math object>

We have $x = [x_1, x_2,x_3,x_4] = [\theta, \phi, \dot{\theta}, \dot{\phi}]$, so $\dot{x} = [\dot{\theta}, \dot{\phi}, \ddot{\theta}, \ddot{\phi}]$. Let's put that in a vector:

In [7]:
theta_ddot_expr = solve_dict[theta_ddot]
phi_ddot_expr = solve_dict[phi_ddot]
state_deriv = Matrix([theta_dot, phi_dot, theta_ddot_expr, phi_ddot_expr])
dotprint(state_deriv)

<IPython.core.display.Math object>

Now we can substitute in our $x_1, x_2$ values in!

In [8]:
# Dictionary for substitutions
subs_dict = {
    theta: state[0],
    phi: state[1],
    theta_dot: state[2],
    phi_dot: state[3]
}

f_expr = state_deriv.subs(subs_dict)
dotprint(f_expr)

<IPython.core.display.Math object>

We now have our state space equations!

## Linearization

The system is already linear, so no linearization is needed.

However, if we want to get the $A,B$ matrices for our linear system in a clean form we can still use Sympy's `jacobian` function.

In [9]:
A = f_expr.jacobian(state)
dotprint(A)

<IPython.core.display.Math object>

In [10]:
B = f_expr.jacobian([tau_theta])
dotprint(B)

<IPython.core.display.Math object>

# Transfer Function

Suppose our outputs are $y = [\theta, \phi]$. What are our $C$ and $D$ matrices?

In [17]:
C = Matrix([[1, 0, 0, 0], [0, 1, 0, 0]])

D = Matrix([[0],[0]])

Next we create our $s$ variable

In [11]:
s = symbols('s')

Now we can create the transfer function.

In [18]:
I = eye(4)
H = (C @ (s*I - A).inv() @ B + D)
H

⎡                     2                               ⎤
⎢              Jₚ⋅Jₛ⋅s  + Jₛ⋅b⋅s + Jₛ⋅k               ⎥
⎢─────────────────────────────────────────────────────⎥
⎢   ⎛       4         3         2         3         2⎞⎥
⎢Jₛ⋅⎝Jₚ⋅Jₛ⋅s  + Jₚ⋅b⋅s  + Jₚ⋅k⋅s  + Jₛ⋅b⋅s  + Jₛ⋅k⋅s ⎠⎥
⎢                                                     ⎥
⎢                    Jₛ⋅b⋅s + Jₛ⋅k                    ⎥
⎢─────────────────────────────────────────────────────⎥
⎢   ⎛       4         3         2         3         2⎞⎥
⎣Jₛ⋅⎝Jₚ⋅Jₛ⋅s  + Jₚ⋅b⋅s  + Jₚ⋅k⋅s  + Jₛ⋅b⋅s  + Jₛ⋅k⋅s ⎠⎦